# 1. Phân tích yêu cầu đề bài

> *Hãy sử dụng dữ liệu được cung cấp trong email này (dữ liệu giá và khối
lượng của một vài mã cổ phiếu) để xây dựng một số mô hình dự đoán biến
động giá cổ phiếu.*
>```
Biến động giá cổ phiếu = Giá cổ phiếu N (phút hoặc giờ hoặc ngày) sau - Giá cổ phiếu hiện tại.
>```
>*Bạn hãy phân tích và tìm mô hình dự đoán biến động giá rồi gửi lại cho Finpros dưới dạng jupyter notebook sau 5 ngày kể từ ngày nhận được
email này.*



- Yêu cầu phân tích biến động giá cổ phiểu => Cập nhật sự biến động theo phút.
- Xu hướng biến động cổ phiếu sẽ có 3 hướng chính: tăng, không đổi hoặc giảm => Sử dụng hàm phân loại softmax để xác định 3 xu hướng trên.
- Dữ liệu dùng cho huấn luyện mô hình có dạng chuỗi thời gian liên tục của giá đóng mở và khối lượng giao dịch =>  sử dụng mô hình LSTM để khai thác tính liên tục của dữ liệu.
- Lựa chọn huấn luyện mô hình kết hợp trên các tập khác nhau và được sắp xếp để không bị xáo trộn về thời gian.

# 2. Tiền xử lí dữ liệu

In [ ]:
# Liên kết với drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## a. Load các thư viện cần thiết

In [27]:
import math, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model

## b. Load dữ liệu đầu vào

In [5]:
os.chdir('/content/drive/MyDrive/LSTM')
FPT = pd.read_csv("FPT.csv")
MSN = pd.read_csv("MSN.csv")
PNJ = pd.read_csv("VIC.csv")
VIC = pd.read_csv("PNJ.csv")

# Hợp nhất các data theo chiều ngang
df = pd.concat([MSN,PNJ,FPT,VIC])

def load(name,df):
  trend = []
  df = df.sort_values(by='Date/Time')
  df['Date/Time'] = pd.to_datetime(df['Date/Time'])  # Chuyển cột Date/Time thành kiểu datetime

  # Tách phần ngày và thời gian
  df['Date'] = df['Date/Time'].dt.date    # Tạo cột ngày
  df['Time'] = df['Date/Time'].dt.time    # Tạo cột thời gian

  # Sắp xếp theo ngày tăng dần và thời gian
  df = df.sort_values(by=['Date', 'Time'], ascending=[True, True])
  df = df.reset_index()

  # Gán nhãn cho dữ liệu dựa vào biến động giá sau khoảng thời gian phút
  trend = []
  def find_trend(future_price, price):
    if future_price < price:
        return 0
    elif future_price == price:
        return 1
    else:
        return 2
  for i in range (len(df)-1):
    future_price = df.at[i+1,'Close']
    price = df.at[i,'Close']
    trend.append(find_trend(future_price, price))
  df['Trend'] = pd.DataFrame(trend)
  df.at[len(df) - 1, 'Trend'] = 1
  df['Trend'] = df['Trend'].astype(int)
  # Xử lý giá trị cuối cùng
  print(name + " dataset: ", len(df))
  print(df[['Date','Time','Close','Volume','Trend']],"\n")
  return df
df_load = load("Data", df)

Data dataset:  459335
              Date      Time  Close  Volume  Trend
0       2017-12-25  09:15:00   73.1    4210      0
1       2017-12-25  09:16:00   73.0    5000      2
2       2017-12-25  09:18:00   73.5     210      0
3       2017-12-25  09:20:00   73.1    2050      0
4       2017-12-25  09:21:00   73.0    1380      2
...            ...       ...    ...     ...    ...
459330  2020-12-22  14:29:00   58.1    2500      1
459331  2020-12-22  14:46:00   58.1   11170      2
459332  2020-12-22  14:46:00  105.7      90      0
459333  2020-12-22  14:46:00   78.3    1500      2
459334  2020-12-22  14:46:00   82.8     200      1

[459335 rows x 5 columns] 



## c. Tiền xử lí dữ liệu

In [6]:
def process_data(name, df, time_step): # time_step định nghĩa số bước nhìn lại quá khứ của mô hình

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[['Close', 'Volume']])

    # Chuyển scaled_data thành DataFrame để dễ thao tác với cột 'Trend'
    scaled_df = pd.DataFrame(scaled_data, columns=['Close', 'Volume'])
    scaled_df['Trend'] = df['Trend'].values

    # Tạo tập dữ liệu huấn luyện
    def create_dataset(data, time_step):
        X, y = [], []
        for i in range(time_step, len(data)):
            X.append(data[i - time_step:i, 0:2])  # Lấy cột 'Close' và 'Volume'
            y.append(int(data[i - time_step, 2]))  # Lấy cột 'Trend'
        return np.array(X), np.array(y)

    # Gọi hàm tạo tập huấn luyện và kiểm tra
    X, y = create_dataset(scaled_df.values, time_step)

    # Chia thành tập huấn luyện và kiểm tra (80% train, 20% test)
    split_index = int(len(X) * 0.8)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Định hình lại X_train và X_test để phù hợp với LSTM
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 2)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 2)

    # Hiển thị thông tin về kích thước dữ liệu
    print(f"{name}: X_train shape: {X_train.shape}, y_train shape: {y_train.shape}, X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = process_data("Scaled data: ",df_load,30)

Scaled data: : X_train shape: (367444, 30, 2), y_train shape: (367444,), X_test shape: (91861, 30, 2), y_test shape: (91861,)


# 3.Xây dựng mô hình LSTM

## a. Khởi tạo kiến trúc mô hình

In [10]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(30, 2)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                        │ (None, 30, 50)              │          10,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 30, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 30, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 30, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 30, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 30, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 30, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 30, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             153 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91,553 (357.63 KB)

 Trainable params: 91,553 (357.63 KB)

 Non-trainable params: 0 (0.00 B)

## b. Huấn luyện mô hình

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=20)
model.save("saved_model.h5")

Epoch 1/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 199s 13ms/step - accuracy: 0.5090 - loss: 0.8453
Epoch 2/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 203s 13ms/step - accuracy: 0.8976 - loss: 0.2241
Epoch 3/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.9248 - loss: 0.1530
Epoch 4/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.9293 - loss: 0.1456
Epoch 5/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.9337 - loss: 0.1385
Epoch 6/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.9360 - loss: 0.1344
Epoch 7/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.9386 - loss: 0.1292
Epoch 8/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 202s 13ms/step - accuracy: 0.9392 - loss: 0.1287
Epoch 9/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 154s 13ms/step - accuracy: 0.9366 - loss: 0.1344
Epoch 10/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 201s 13ms/step - accuracy: 0.9411 - loss: 0.1261
Epoch 11/20
11483/11483 ━━━━━━━━━━━━━━━━━━━━ 203s 13ms/step - accurac

# 4. Đánh giá mô hình

In [21]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
def map_label_to_trend(label):
    if label == 2:
        return 'Tăng'
    elif label == 0:
        return 'Giảm'
    elif label == 1:
        return 'Không đổi'

predicted_trends = [map_label_to_trend(label) for label in predicted_labels]
test = pd.DataFrame({
    'Giá trị dự đoán': predicted_labels,
    'Xu hướng dự đoán': predicted_trends,
    'Giá trị thực': y_test
})
print(test)



2871/2871 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step
       Giá trị dự đoán Xu hướng dự đoán  Giá trị thực
0                    0             Giảm             0
1                    2             Tăng             2
2                    0             Giảm             0
3                    2             Tăng             2
4                    2             Tăng             2
...                ...              ...           ...
91856                0             Giảm             0
91857                2             Tăng             2
91858                0             Giảm             0
91859                2             Tăng             2
91860                0             Giảm             0

[91861 rows x 3 columns]


In [29]:
# Đánh giá qua ma trận hỗn loạn (confusion matrix)
cfm = confusion_matrix(y_test, predicted_labels)
print(cfm)

[[43803   638     0]
 [    0  3043     0]
 [    0   863 43514]]


# 5. Kết luận
- Xây dựng được mô hình dự đoán biến động giá cổ phiếu với độ chính xác tương đối cao (0.9522)
- Kết quả đánh giá cho thấy mô hình đạt hiệu suất cao khi thực hiện so sánh giá trị dự đoán và giá trị thực trên tập test.
- Kết quả confusion matrix cho thấy khả năng dự đoán biến động đúng với thực tế của mô hình cao (các số trên đường chéo trái lớn).